In [2]:
%run 'Setup.py'

8 different classes: Electronic, Experimental, Folk, Hip-Hop, Instrumental, International, Pop or Rock.
objective 1: construct a classifier which, based on the features of a song, predicts its genre
objective 2: estimate its generalisation error under the 0–1 loss.
Features are real-valued, correspond to summary statistics (mean, sd, skewness, kurtosis, median, min, max) of 
time series of various music features, such as the chromagram or the Mel-frequency cepstrum.
Feature description: 

Feature description: 
chroma_cens: Chroma Energy Normalized (CENS, 12 chroma) - 84 features
chroma_cqt: Constant-Q chromagram (12 chroma) - 84 features
chroma_stft: Chromagram (12 chroma) - 84 features
mfcc: Mel-frequency cepstrum (20 coefficients) - 140 features
rmse: Root-mean-square - 7 features
spectral_bandwidth: Spectral bandwidth - 7 features
spectral_centroid: Spectral centroid - 7 features
spectral_contrast: Spectral contrast (7 frequency bands) - 49 features
spectral_rolloff: Roll-off freque

In [3]:
# Prepare data
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_np.ravel()) #

# Split training data into training and temporary validation sets
X_train, X_temp, Y_train, Y_temp = train_test_split(x_train, y_train_encoded, test_size=0.4, random_state=42)

# Split the temporary validation set into validation and fake test set
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)  
X_real_test_scaled = scaler.transform(x_test) # real test set we don't have labels for

In [4]:
def split_features_by_type(X, feature_structure):
    """
    Splits the dataset into subsets based on the feature structure provided.

    :param X: numpy array, the dataset to be split (features only)
    :param feature_structure: dict, keys are feature names and values are the number of features of that type
    :return: dict of feature subsets
    """
    feature_subsets = {}
    start_idx = 0
    
    for feature_name, feature_count in feature_structure.items():
        end_idx = start_idx + feature_count
        feature_subsets[feature_name] = X[:, start_idx:end_idx]
        start_idx = end_idx
    
    return feature_subsets

# Define the feature structure
feature_structure = {
    'chroma_cens': 84,
    'chroma_cqt': 84,
    'chroma_stft': 84,
    'mfcc': 140,
    'rmse': 7,
    'spectral_bandwidth': 7,
    'spectral_centroid': 7,
    'spectral_contrast': 49,
    'spectral_rolloff': 7,
    'tonnetz': 42,
    'zcr': 7
}

# Boosting separately on feature subsets

Load Data and Convert to DMatrix

In [12]:
train_subsets = split_features_by_type(X_train_scaled, feature_structure)
val_subsets = split_features_by_type(X_val_scaled, feature_structure)
test_subsets = split_features_by_type(X_test_scaled, feature_structure)

In [13]:
def objective(trial, X_sub, Y_sub):
    # Hyperparameters
    params = {
        'objective': 'multi:softmax',
        'num_class': 8,
        'max_depth': trial.suggest_int('max_depth', 3, 60),
        'eta': trial.suggest_float('eta', 0.01, 0.4),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'eval_metric': 'merror'  # Multiclass Classification Error
    }

    # Convert the subset dataset into DMatrix form
    dmatrix = xgb.DMatrix(X_sub, label=Y_sub)

    # Perform cross-validation
    cv_results = xgb.cv(params, dmatrix, num_boost_round=5000, nfold=5, stratified=True,
                        early_stopping_rounds=25, seed=42, verbose_eval=False)

    # Extract the minimum mean merror from the CV results
    min_mean_merror = cv_results['test-merror-mean'].min()

    return min_mean_merror

In [14]:
best_params_subsets = {}
validation_accuracies = {}

for feature_name, feature_count in feature_structure.items():
    print(f"Running study for feature subset: {feature_name}")
    
    # Prepare the data for this subset
    X_sub_train = train_subsets[feature_name]
    Y_sub_train = Y_train  # Y_train should be defined in your context

    def subset_objective(trial):
        return objective(trial, X_sub_train, Y_sub_train)

    study = optuna.create_study(direction='minimize', study_name=f"XGB_{feature_name}")
    study.optimize(subset_objective, n_trials=100)

    best_params_subsets[feature_name] = study.best_trial.params

    # # Merge train and validation subsets for final model retraining
    # X_sub_train_val_combined = np.vstack((X_sub_train, val_subsets[feature_name]))
    # Y_train_val_combined = np.concatenate((Y_train, Y_val))  # Assuming Y_val is defined
    # 
    # # Retrain the model on the combined training and validation set with the best parameters
    # dtrain_val_combined = xgb.DMatrix(X_sub_train_val_combined, label=Y_train_val_combined)
    # Retrain model with optimal parameters with more boosting rounds
    params = {
        'objective': 'multi:softmax',
        'num_class': 8,
        **best_params_subsets[feature_name],  # Unpack the best parameters
    }
    final_model = xgb.train(params, X_sub_train, num_boost_round=10_000) # dtrain_val_combined

    # Evaluate the final model on the validation set
    dval = xgb.DMatrix(val_subsets[feature_name], label=Y_val)
    preds = final_model.predict(dval)
    val_accuracy = accuracy_score(Y_val, preds)
    validation_accuracies[feature_name] = val_accuracy

    # Save the final model
    model_name = f'Models/XGBoost-Feature-Subsets/xgboost_{feature_name}_final.model'
    final_model.save_model(model_name)

    print(f"Validation accuracy for {feature_name}: {val_accuracy}")
    

[I 2024-03-13 23:14:32,121] A new study created in memory with name: XGB_chroma_cens


Running study for feature subset: chroma_cens


[W 2024-03-13 23:15:03,961] Trial 0 failed with parameters: {'max_depth': 16, 'eta': 0.18260583038077527, 'subsample': 0.8210097791906202, 'colsample_bytree': 0.8412711335976696} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_278/2774506645.py", line 12, in subset_objective
    return objective(trial, X_sub_train, Y_sub_train)
  File "/tmp/ipykernel_278/3102675667.py", line 17, in objective
    cv_results = xgb.cv(params, dmatrix, num_boost_round=5000, nfold=5, stratified=True,
  File "/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/xgboost/training.py", line 538, in cv
    booster.update(i, obj)
  File "/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/xgboost/training.py", line 229, in update
    fold.update(iteration, obj)
  File "/opt/saturncloud/envs/

KeyboardInterrupt: 

## Save Best Parameters 

In [ ]:
import json
from datetime import datetime
# Format the current date and time as a string
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Filenames with date and time
filename_best_params = f"best_params_subsets_{timestamp}.json"
filename_validation_accuracies = f"validation_accuracies_{timestamp}.json"

# Save best_params_subsets
with open(filename_best_params, 'w') as file:
    json.dump(best_params_subsets, file, indent=4)

# Save validation_accuracies
with open(filename_validation_accuracies, 'w') as file:
    json.dump(validation_accuracies, file, indent=4)


## Add, Commit, Push Tuning Results

In [ ]:
!git add .
!git commit -m "tuning completed"
!git push origin HEAD


In [5]:
len(feature_structure)

11

In [6]:
print(feature_structure)

{'chroma_cens': 84, 'chroma_cqt': 84, 'chroma_stft': 84, 'mfcc': 140, 'rmse': 7, 'spectral_bandwidth': 7, 'spectral_centroid': 7, 'spectral_contrast': 49, 'spectral_rolloff': 7, 'tonnetz': 42, 'zcr': 7}
